In [55]:
import csv
from pathlib import Path
import pandas as pd
from pandas import DataFrame
import os
from dotenv import load_dotenv
from datetime import datetime
from datetime import date
from datetime import timedelta
import requests
import json

import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
import panel as pn
from panel.interact import interact
from panel import widgets
import numpy as np

%matplotlib inline

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

# Input Portfolio
Read in the input portfolio CSV file and output a portfolio dataframe
* Inputs:
    * Portfolio CSV file
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares
    * Assumptions: Date unsorted, no '$' sign, no commas
* Outputs:
    * Portfolio database
    * Format: Ticker, Transaction (B=buy, S=Sell), Date, Transaction Price, Number of Shares, Transaction Cost
    * Database sorted by ascending date

In [56]:
# Set the path for the CSV file
input_file_path = Path("TestPortfolio.csv")

In [57]:
#Create empty list to store row data from CSV
portfolio_transactions = []

In [58]:
#Open the CSV file; read in row data; calculate transaction cost per transacation
with open(input_file_path, 'r') as csvfile:
    #Set reader object
    csvreader = csv.reader(csvfile, delimiter = ',')
 
    #Read the header
    csv_header = next(csvreader)
    
    #Create new column
    csv_header.append("Transaction Cost")
    portfolio_transactions.append(csv_header)

    #Iterate through the data 
    for row in csvreader:
        #Calculate transaction cost
        cost = float(row[3]) * float(row[4])
        
        #Append transaction cost to list
        row.append(cost)
        portfolio_transactions.append(row)

In [59]:
#Convert transaction list to dataframe
portfolio_df = pd.DataFrame(portfolio_transactions, columns=['Ticker', 'Transaction', 'Date', 'Transaction Price', 'Number of shares', 'Transaction Cost'])
#Drop first row (duplicate header)
portfolio_df=portfolio_df.drop(portfolio_df.index[0])
portfolio_df

,Ticker,Transaction,Date,Transaction Price,Number of shares,Transaction Cost
1,AAPL,B,3/5/12,77.8,100,7780
2,AAPL,B,4/1/13,60.57,100,6057
3,TSLA,B,9/23/13,189.9,100,18990
4,TSLA,S,11/18/13,122.38,75,9178.5
5,TSLA,B,5/9/16,208,30,6240
6,GOOG,B,8/7/17,915.39,10,9153.9
7,AAPL,S,3/16/20,240,50,12000


In [60]:
#Convert Date to datetime and set as index
portfolio_df['Date']=pd.to_datetime(portfolio_df['Date'])
portfolio_df.set_index(['Date'], inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


In [61]:
# Sort dataframe in ascending order
portfolio_df.sort_index(inplace=True)
portfolio_df

,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
Date,,,,,
2012-03-05,AAPL,B,77.8,100,7780
2013-04-01,AAPL,B,60.57,100,6057
2013-09-23,TSLA,B,189.9,100,18990
2013-11-18,TSLA,S,122.38,75,9178.5
2016-05-09,TSLA,B,208,30,6240
2017-08-07,GOOG,B,915.39,10,9153.9
2020-03-16,AAPL,S,240,50,12000


# Pull Stock Data
Using the portfolio database, determine the number of unique stocks and pull the stock history.
* Inputs:
    * Portfolio database (portfolio_df)
* Outputs:
    * Ticker Stat database (ticker_stats_df) (Tickers as Rows)
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
    * Dailiy Closing Price database (closing_df)

In [62]:
# Set environment variables from the .env file
load_dotenv()

True

In [63]:
# Grab API Key for https://financialmodelingprep.com
stock_key = os.getenv("FMP_API_KEY")

In [64]:
# Create empty list for stock tickers
stock_list=[]

In [65]:
# Determine unique stocks
group=portfolio_df.groupby('Ticker')
# Store unique stocks to stock list
group.groups.keys()
stock_list=list(group.groups)
stock_list

['AAPL', 'GOOG', 'TSLA']

In [66]:
#Create new dataframe resetting date index
portfolio2_df=portfolio_df.reset_index()
portfolio2_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost
0,2012-03-05,AAPL,B,77.8,100,7780
1,2013-04-01,AAPL,B,60.57,100,6057
2,2013-09-23,TSLA,B,189.9,100,18990
3,2013-11-18,TSLA,S,122.38,75,9178.5
4,2016-05-09,TSLA,B,208,30,6240
5,2017-08-07,GOOG,B,915.39,10,9153.9
6,2020-03-16,AAPL,S,240,50,12000


In [67]:
#Find the min dates per tikcer
min_stock_dates=portfolio2_df.groupby('Ticker').min()
min_stock_dates

,Date,Transaction,Transaction Price,Number of shares,Transaction Cost
Ticker,,,,,
AAPL,2012-03-05,B,240,100,6057.0
GOOG,2017-08-07,B,915.39,10,9153.9
TSLA,2013-09-23,B,122.38,100,6240.0


In [68]:
#Store yesterday's date
yesterday = date.today() - timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
yesterday

'2020-06-15'

In [69]:
#Define Main Loop Variables
stock_dict={}            # Create dictionary to store stock data
count=0                  # Create counter as index for looping through stock list

#For each ticker in the stock list, pull the stock data
for ticker in stock_list:
    #Define new dictionary key set per ticker
    stock_dict[ticker]={
        'PE Ratio'            : 0,
        'Profit Margin'       : 0,
        'Cash'                : 0,
        'Beta'                : 0,
        'Last Dividend'       : 0,
        'Industry'            : 0,
        'Sector'              : 0,
        'Daily Closing Price' : 0,
        }
    
######
    #Get Beta, Last Div, Industry and Sector
    url = f"https://financialmodelingprep.com/api/v3/profile/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    #Store stock data in dictionary
    stock_dict[ticker]['Beta']=parsed[0]['beta']
    stock_dict[ticker]['Last Dividend']=parsed[0]['lastDiv']
    stock_dict[ticker]['Industry']=parsed[0]['industry']
    stock_dict[ticker]['Sector']=parsed[0]['sector']
    

######
    # Grab PE Ratio
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store PE Ratio in dictionary
    stock_dict[ticker]['PE Ratio']=parsed[0]['peRatio']
    
    
######    
    # Grab Profit Margin
    url = f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Profit Margin']=parsed[0]['netProfitMargin']
    
   
###### 
    # Grab Cash
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
    
    # Store Profit Margin in dictionary
    stock_dict[ticker]['Cash']=parsed[0]['cashAndCashEquivalents']

     
######
    #Grab the first purchase date for current ticker
    datetime_obj=datetime.strptime(str(min_stock_dates['Date'][ticker]),"%Y-%m-%d %H:%M:%S")
    date_obj=datetime_obj.date()
    first_purchase_date=str(date_obj)
    
    # Grab the Daily Closing Prices between the first purchase date and yesterday
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={first_purchase_date}&to={yesterday}&apikey={stock_key}"
    requests.get(url).content
    parsed = json.loads(requests.get(url).content)
 
    ### Daily Stock Pull Variables
    count2=0                 # Create counter as index for looping through dictionary of stock data    
    daily_dict={}            # Create empty dictionary to store daily stock data before appending to daily_list_of_dict
    daily_list_of_dict = []  # Create empty list to store dictionary info from daily_dict before storing to stock_dict

    # Loop through historical data and grab daily closing date and price
    for i in parsed['historical']:
        daily_dict['Date'] = parsed['historical'][count2]['date']
        daily_dict['Price'] = parsed['historical'][count2]['close']
        daily_list_of_dict.append(daily_dict.copy())
        count2+=1

    #Store historical stock data to stock_dict dictionary
    stock_dict[ticker]['Daily Closing Price']=daily_list_of_dict
    
    
    #increment counter
    count+=1 

In [70]:
#Copy stock_dict so that main loop above doesn't have to be run multiple times
stock_dict_copy = stock_dict
stock_dict_copy

{'AAPL': {'PE Ratio': 20.81351450883162,
  'Profit Margin': 0.21238094505984456,
  'Cash': 48844000000,
  'Beta': 1.228499,
  'Last Dividend': 3.08,
  'Industry': 'Consumer Electronics',
  'Sector': 'Technology',
  'Daily Closing Price': [{'Date': '2020-06-15', 'Price': 342.99},
   {'Date': '2020-06-12', 'Price': 338.8},
   {'Date': '2020-06-11', 'Price': 335.9},
   {'Date': '2020-06-10', 'Price': 352.84},
   {'Date': '2020-06-09', 'Price': 343.99},
   {'Date': '2020-06-08', 'Price': 333.46},
   {'Date': '2020-06-05', 'Price': 331.5},
   {'Date': '2020-06-04', 'Price': 322.32},
   {'Date': '2020-06-03', 'Price': 325.12},
   {'Date': '2020-06-02', 'Price': 323.34},
   {'Date': '2020-06-01', 'Price': 321.85},
   {'Date': '2020-05-29', 'Price': 317.94},
   {'Date': '2020-05-28', 'Price': 318.25},
   {'Date': '2020-05-27', 'Price': 318.11},
   {'Date': '2020-05-26', 'Price': 316.73},
   {'Date': '2020-05-22', 'Price': 318.89},
   {'Date': '2020-05-21', 'Price': 316.85},
   {'Date': '2020-0

In [71]:
# Create DataFrame showing Beta, P/E Ratio, Profit Margin, 
# Cash, and current Dividends per ticker
ticker_stats_df=DataFrame(stock_dict_copy)
ticker_stats_df.drop('Daily Closing Price', inplace=True)
ticker_stats_df

,AAPL,GOOG,TSLA
PE Ratio,20.8135,14.6956,-131.582
Profit Margin,0.212381,0.212181,-0.035072
Cash,48844000000,18498000000,6268000000
Beta,1.2285,1.02276,0.580101
Last Dividend,3.08,0,0
Industry,Consumer Electronics,Internet Content & Information,Auto Manufacturers
Sector,Technology,Communication Services,Consumer Cyclical


In [72]:
# Create a transpose of ticker_stats_df
ticker_stats_df_T=ticker_stats_df.T
ticker_stats_df_T=ticker_stats_df_T[['PE Ratio', 'Profit Margin', 'Cash', 'Beta', 'Last Dividend', 'Industry', 'Sector']]
ticker_stats_df_T

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.8135,0.212381,48844000000,1.2285,3.08,Consumer Electronics,Technology
GOOG,14.6956,0.212181,18498000000,1.02276,0,Internet Content & Information,Communication Services
TSLA,-131.582,-0.035072,6268000000,0.580101,0,Auto Manufacturers,Consumer Cyclical


In [73]:
# Closing Data

In [74]:
# Create DataFrame showing Daily Closing Price per ticker
daily_closing_df=DataFrame(stock_dict_copy)
daily_closing_df.drop(['Beta', 'Cash', 'Industry', 'Last Dividend', 'PE Ratio', 'Profit Margin', 'Sector'], inplace=True)
daily_closing_df

,AAPL,GOOG,TSLA
Daily Closing Price,"[{'Date': '2020-06-15', 'Price': 342.99}, {'Da...","[{'Date': '2020-06-15', 'Price': 1419.85}, {'D...","[{'Date': '2020-06-15', 'Price': 990.9}, {'Dat..."


In [75]:
aapl_df = pd.DataFrame(daily_closing_df['AAPL'][0])
#aapl_df.rename(columns={'Date' : 'AAPL Closing Date', 'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.rename(columns={'Price' : 'AAPL Closing Price'}, inplace=True)
aapl_df.head()

,Date,AAPL Closing Price
0,2020-06-15,342.99
1,2020-06-12,338.80
2,2020-06-11,335.90
3,2020-06-10,352.84
4,2020-06-09,343.99


In [76]:
goog_df = pd.DataFrame(daily_closing_df['GOOG'][0])
#goog_df.rename(columns={'Date' : 'GOOG Closing Date', 'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.rename(columns={'Price' : 'GOOG Closing Price'}, inplace=True)
goog_df.head()

,Date,GOOG Closing Price
0,2020-06-15,1419.85
1,2020-06-12,1413.18
2,2020-06-11,1403.84
3,2020-06-10,1465.85
4,2020-06-09,1456.16


In [77]:
tsla_df = pd.DataFrame(daily_closing_df['TSLA'][0])
#tsla_df.rename(columns={'Date' : 'TSLA Closing Date', 'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.rename(columns={'Price' : 'TSLA Closing Price'}, inplace=True)
tsla_df.head()

,Date,TSLA Closing Price
0,2020-06-15,990.90
1,2020-06-12,935.28
2,2020-06-11,972.84
3,2020-06-10,1025.05
4,2020-06-09,940.67


In [78]:
closing_df = aapl_df.set_index('Date').join(goog_df.set_index('Date')).join(tsla_df.set_index('Date'))
closing_df.head()

,AAPL Closing Price,GOOG Closing Price,TSLA Closing Price
Date,,,
2020-06-15,342.99,1419.85,990.90
2020-06-12,338.80,1413.18,935.28
2020-06-11,335.90,1403.84,972.84
2020-06-10,352.84,1465.85,1025.05
2020-06-09,343.99,1456.16,940.67


# Insert Alex's Code

In [79]:
# Convert daily closing price data (stock_dict_copy['Daily Closing Price'])
# into a new dataframe (closing_price_df) with only Date, Ticker, and Close columns
date=[]
price=[]
stock_name=[]

for stock in stock_dict_copy:
    for d in stock_dict_copy[stock]['Daily Closing Price']:
        stock_name.append(stock)
        date.append(datetime.strptime(d['Date'],'%Y-%m-%d'))
        price.append(d['Price'])
        
closing_price_df= pd.DataFrame(list(zip(date, stock_name, price)), columns=['Date', 'Ticker', 'Close'])
closing_price_df

,Date,Ticker,Close
0,2020-06-15,AAPL,342.99
1,2020-06-12,AAPL,338.80
2,2020-06-11,AAPL,335.90
3,2020-06-10,AAPL,352.84
4,2020-06-09,AAPL,343.99
...,...,...,...
4492,2013-09-27,TSLA,190.90
4493,2013-09-26,TSLA,188.64
4494,2013-09-25,TSLA,185.24
4495,2013-09-24,TSLA,182.33


In [80]:
closing_price_df=closing_price_df.sort_values('Date')
closing_price_df

,Date,Ticker,Close
2083,2012-03-05,AAPL,76.17
2082,2012-03-06,AAPL,75.75
2081,2012-03-07,AAPL,75.81
2080,2012-03-08,AAPL,77.43
2079,2012-03-09,AAPL,77.88
...,...,...,...
1,2020-06-12,AAPL,338.80
2804,2020-06-12,TSLA,935.28
2803,2020-06-15,TSLA,990.90
2084,2020-06-15,GOOG,1419.85


In [81]:
#Merge the input portfolio dataframe to the closing price dataframe
portfolio3_df = pd.merge(portfolio_df, closing_price_df, how='right', left_on=['Date', 'Ticker'], right_on=['Date', 'Ticker'])
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
1,2013-04-01,AAPL,B,60.57,100,6057,61.27
2,2013-09-23,TSLA,B,189.9,100,18990,181.11
3,2013-11-18,TSLA,S,122.38,75,9178.5,121.58
4,2016-05-09,TSLA,B,208,30,6240,208.92
...,...,...,...,...,...,...,...
4492,2020-06-12,AAPL,NaN,NaN,NaN,NaN,338.80
4493,2020-06-12,TSLA,NaN,NaN,NaN,NaN,935.28
4494,2020-06-15,TSLA,NaN,NaN,NaN,NaN,990.90
4495,2020-06-15,GOOG,NaN,NaN,NaN,NaN,1419.85


In [82]:
portfolio3_df=portfolio3_df.sort_values('Date')
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
7,2012-03-06,AAPL,NaN,NaN,NaN,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,NaN,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,NaN,NaN,77.43
10,2012-03-09,AAPL,NaN,NaN,NaN,NaN,77.88
...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,NaN,NaN,1413.18
4493,2020-06-12,TSLA,NaN,NaN,NaN,NaN,935.28
4495,2020-06-15,GOOG,NaN,NaN,NaN,NaN,1419.85
4494,2020-06-15,TSLA,NaN,NaN,NaN,NaN,990.90


In [83]:
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].fillna(0)
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close
0,2012-03-05,AAPL,B,77.8,100,7780,76.17
7,2012-03-06,AAPL,NaN,NaN,0,NaN,75.75
8,2012-03-07,AAPL,NaN,NaN,0,NaN,75.81
9,2012-03-08,AAPL,NaN,NaN,0,NaN,77.43
10,2012-03-09,AAPL,NaN,NaN,0,NaN,77.88
...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,0,NaN,1413.18
4493,2020-06-12,TSLA,NaN,NaN,0,NaN,935.28
4495,2020-06-15,GOOG,NaN,NaN,0,NaN,1419.85
4494,2020-06-15,TSLA,NaN,NaN,0,NaN,990.90


In [84]:
# Convert Number of shares and Transaction Price to float
portfolio3_df['Number of shares']=portfolio3_df['Number of shares'].astype(float)
portfolio3_df['Transaction Price']=portfolio3_df['Transaction Price'].astype(float)

In [85]:
# Calculate the net number of shares and total number of shares
portfolio3_df['Net Shares'] = portfolio3_df.apply(lambda row: row['Number of shares'] if row.Transaction == "B" else row['Number of shares'] * -1, axis=1)
portfolio3_df['Total Shares'] = portfolio3_df[['Net Shares','Ticker']].groupby('Ticker').cumsum()
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0
...,...,...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,0.0,NaN,1413.18,-0.0,10.0
4493,2020-06-12,TSLA,NaN,NaN,0.0,NaN,935.28,-0.0,55.0
4495,2020-06-15,GOOG,NaN,NaN,0.0,NaN,1419.85,-0.0,10.0
4494,2020-06-15,TSLA,NaN,NaN,0.0,NaN,990.90,-0.0,55.0


In [86]:
# Calculate cum Daily Balances based on Total Shares and daily closing price
portfolio3_df['Daily Balance'] = portfolio3_df['Close']*portfolio3_df['Total Shares']
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,7617.0
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,7575.0
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,7581.0
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,7743.0
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,7788.0
...,...,...,...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,0.0,NaN,1413.18,-0.0,10.0,14131.8
4493,2020-06-12,TSLA,NaN,NaN,0.0,NaN,935.28,-0.0,55.0,51440.4
4495,2020-06-15,GOOG,NaN,NaN,0.0,NaN,1419.85,-0.0,10.0,14198.5
4494,2020-06-15,TSLA,NaN,NaN,0.0,NaN,990.90,-0.0,55.0,54499.5


In [87]:
portfolio3_df['Date']

0      2012-03-05
7      2012-03-06
8      2012-03-07
9      2012-03-08
10     2012-03-09
          ...    
4491   2020-06-12
4493   2020-06-12
4495   2020-06-15
4494   2020-06-15
4496   2020-06-15
Name: Date, Length: 4497, dtype: datetime64[ns]

In [88]:
portfolio3_df['Cumm Contribution'] = pd.DataFrame(portfolio3_df['Transaction Price'] * portfolio3_df['Net Shares'])
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance,Cumm Contribution
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,7617.0,7780.0
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,7575.0,NaN
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,7581.0,NaN
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,7743.0,NaN
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,7788.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,0.0,NaN,1413.18,-0.0,10.0,14131.8,NaN
4493,2020-06-12,TSLA,NaN,NaN,0.0,NaN,935.28,-0.0,55.0,51440.4,NaN
4495,2020-06-15,GOOG,NaN,NaN,0.0,NaN,1419.85,-0.0,10.0,14198.5,NaN
4494,2020-06-15,TSLA,NaN,NaN,0.0,NaN,990.90,-0.0,55.0,54499.5,NaN


In [89]:
portfolio3_df['Gains/Losses'] = portfolio3_df['Daily Balance'] - portfolio3_df['Transaction Cost']
portfolio3_df

,Date,Ticker,Transaction,Transaction Price,Number of shares,Transaction Cost,Close,Net Shares,Total Shares,Daily Balance,Cumm Contribution,Gains/Losses
0,2012-03-05,AAPL,B,77.8,100.0,7780,76.17,100.0,100.0,7617.0,7780.0,-163
7,2012-03-06,AAPL,NaN,NaN,0.0,NaN,75.75,-0.0,100.0,7575.0,NaN,NaN
8,2012-03-07,AAPL,NaN,NaN,0.0,NaN,75.81,-0.0,100.0,7581.0,NaN,NaN
9,2012-03-08,AAPL,NaN,NaN,0.0,NaN,77.43,-0.0,100.0,7743.0,NaN,NaN
10,2012-03-09,AAPL,NaN,NaN,0.0,NaN,77.88,-0.0,100.0,7788.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4491,2020-06-12,GOOG,NaN,NaN,0.0,NaN,1413.18,-0.0,10.0,14131.8,NaN,NaN
4493,2020-06-12,TSLA,NaN,NaN,0.0,NaN,935.28,-0.0,55.0,51440.4,NaN,NaN
4495,2020-06-15,GOOG,NaN,NaN,0.0,NaN,1419.85,-0.0,10.0,14198.5,NaN,NaN
4494,2020-06-15,TSLA,NaN,NaN,0.0,NaN,990.90,-0.0,55.0,54499.5,NaN,NaN


In [90]:
portfolio3_df_returns = portfolio3_df.sort_values(['Ticker', 'Date'])


In [91]:
portfolio_df_returns = portfolio3_df.drop(columns=['Transaction','Transaction Price', 'Number of shares', 'Transaction Cost', 'Close', 'Net Shares', 'Total Shares'])
portfolio_df_returns.drop(columns=['Cumm Contribution', 'Gains/Losses'], inplace=True)

In [92]:
portfolio_df_returns['Returns'] = pd.DataFrame(portfolio_df_returns['Daily Balance'].pct_change())
portfolio_df_returns

,Date,Ticker,Daily Balance,Returns
0,2012-03-05,AAPL,7617.0,NaN
7,2012-03-06,AAPL,7575.0,-0.005514
8,2012-03-07,AAPL,7581.0,0.000792
9,2012-03-08,AAPL,7743.0,0.021369
10,2012-03-09,AAPL,7788.0,0.005812
...,...,...,...,...
4491,2020-06-12,GOOG,14131.8,-0.721924
4493,2020-06-12,TSLA,51440.4,2.640046
4495,2020-06-15,GOOG,14198.5,-0.723982
4494,2020-06-15,TSLA,54499.5,2.838398


In [93]:
portfolio_df_returns.drop(columns=['Daily Balance'], inplace=True)

In [94]:
portfolio_df_returns.sort_values('Ticker')

,Date,Ticker,Returns
0,2012-03-05,AAPL,NaN
2095,2017-02-08,AAPL,0.003877
2097,2017-02-09,AAPL,0.788734
2098,2017-02-10,AAPL,-0.002266
2101,2017-02-13,AAPL,0.800286
...,...,...,...
2399,2017-09-01,TSLA,1.085369
702,2014-05-05,TSLA,-0.684610
2403,2017-09-05,TSLA,-0.413976
2389,2017-08-29,TSLA,0.004918


In [95]:
def returns_plot(df):
    return df.hvplot.line(
        x="Date",
        y="Daily Balance",
        title="Returns",
        groupby= "Ticker")

returns_plot(portfolio3_df)

:DynamicMap   [Ticker]
   :Curve   [Date]   (Daily Balance)

In [96]:
def pct_change_plot(df):
    return df.hvplot.line(
        x="Date",
        y="Returns",
        title="Returns Daily Percent Change",
        groupby= "Ticker")

pct_change_plot(portfolio_df_returns)

:DynamicMap   [Ticker]
   :Curve   [Date]   (Returns)

# Insert Heena's Code

# Create Ticker Stats Panel
Using the portfolio database, determine the number of unique stocks and pull stock history.
* Inputs:
    * Transposed Ticker Stat Database (ticker_stats_df_T) (Tickers as Columns)
* Outputs:
    * Ticker Stat Panel (stats_panel)

In [25]:
# Define function that will change negative numbers to red text
def color_negative_red(value):
    if value < 0:
        color='red'
    else:
        color='black'
    return 'color: %s' % color

# Define function that will highlight the max value
def highlight_max(value):
    is_max = value == value.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [26]:
#Format ticker_stats_df_T to:
# Highlight negative values red
# Highlight larget value yellow
# Add '$' and ',' to currency
# Change percentages to %
# Change floats to 2 decimal places
formatted_stats_df = ticker_stats_df_T.style.applymap(
    color_negative_red, subset=['Beta', 'Cash', 'PE Ratio', 'Profit Margin']
    ).apply(
    highlight_max, subset=['Beta', 'Cash', 'Last Dividend', 'PE Ratio', 'Profit Margin']
    ).format(
    {'Beta':'{:,.2f}', 'Cash':'${:,}', 'Last Dividend':'${:,.2f}', 'PE Ratio':'{:,.2f}', 'Profit Margin':'{:,.1%}'}
    )
formatted_stats_df

,PE Ratio,Profit Margin,Cash,Beta,Last Dividend,Industry,Sector
AAPL,20.81,21.2%,"$48,844,000,000",1.23,$3.08,Consumer Electronics,Technology
GOOG,14.70,21.2%,"$18,498,000,000",1.02,$0.00,Internet Content & Information,Communication Services
TSLA,-131.58,-3.5%,"$6,268,000,000",0.58,$0.00,Auto Manufacturers,Consumer Cyclical


In [27]:
stats_panel = pn.panel(formatted_stats_df)
stats_panel

HTML(Styler)

In [ ]:
# code below is my scratchpad area, ignore

In [ ]:
for row in pf:
    if row[index] == 'B':
        lll
    else:
        lll
return lll

In [137]:
stock_dict_copy2=pd.DataFrame(stock_dict_copy)
stock_dict_copy2.to_csv('stock_dict_copy.csv')